## Building Machine Learning Classifiers: Building a basic random forest model

### Read in and clean the text

In [10]:
import nltk
import pandas as pd
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data  = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None)
data.columns = ['Label', 'body_text']
data.head()

,Label,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [12]:
#creating a new feature
def count_puct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(' ')),3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(' '))
data['punct%']= data['body_text'].apply(lambda x: count_puct(x))

#removing punctuations, stopwords, applying tokenization and stemming
def clean_text(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokenize = re.split('\W+', text)
    text = [word for word in tokenize if word not in stopwords]
    text = [ps.stem(word) for word in text]
    return text
    

In [14]:
#converting the text part of the data to vectors using tfidf
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['body_text'])

#concatinating it back to the numerical featres
features = pd.concat([data['body_len'],data['punct%'], pd.DataFrame(x_tfidf.toarray())], axis=1)
features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'n_features_', 'predict', 'predict_log_proba', 'predict_proba', 'score',

### Explore RandomForestClassifier through Cross_validation

In [17]:
from sklearn.model_selection import KFold, cross_val_score

In [20]:
rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, features, data['Label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97396768, 0.97935368, 0.97666068, 0.96495957, 0.97484277])